# Datenbezug über SRU und Umwandlung der Inhalte in eine Tabelle:

## Grundlagen zur SRU-Schnittstelle der DNB:

Die DNB bietet Ihre Daten auf drei verschiedene "Kataloge" gesplittet an, von denen immer einer für eine Abfrage ausgewählt werden muss. Dies geschieht über eine Erweiterung der Base-URL. Zur Verfügung stehen folgende Kataloge:

- Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
- Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
- Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die erweiterungen für die URL sind folgende:

- DNB: https://services.dnb.de/sru/dnb
- DMA: https://services.dnb.de/sru/dnb.dma
- GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine "Explain-Response" in XML zurück. 
Mit Hilfe der Bibliothek BeautifulSoup kann die Antwort direkt in XML umgewandelt werden.

[Weitere Informationen und Dokumentation zur SRU-Schnittstelle](https://www.dnb.de/sru)

### Laden benötigter Zusatzbibliotheken: 

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


## SRU-Anfrage

In [3]:
#Base-URL für den  Katalog der DNB (Titeldaten):
dnb = requests.get("https://services.dnb.de/sru/dnb")

#Einfache Amfrage an die Schnittstelle ohne Suche 
response = soup(dnb.content, features="xml")  #Parsen der Schnittstellenantwort als xml
print(response.prettify()[0:500])  #Einschränken der Ausgabe auf die ersten 500 Zeichen

<?xml version="1.0" encoding="utf-8"?>
<explainResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <record>
  <recordSchema>
   http://explain.z3950.org/dtd/2.0/
  </recordSchema>
  <recordPacking>
   xml
  </recordPacking>
  <recordData>
   <ns:explain id="Deutsche Nationalbibliothek" xmlns:ns="http://explain.z3950.org/dtd/2.0/">
    <ns:serverInfo protocol="sru" version="1.1">
     <ns:host>
      services.dnb.de
     </ns:host>
     <ns:port>
      443
     </ns:port>


Für eine Suchanfrage an die Daten der DNB wird nun zunächst über die Wahl der URL der Katalog definiert. Mit Hilfe der Variable parameter werdem dann alle weiteren Parameter, die die SRU-Schnittstelle benötigt, übergeben.

In [5]:
#Base-URL:
dnb_url = "https://services.dnb.de/sru/dnb"

#Parameter, die mit der Anfrage übergeben werden sollen: 
parameter = {'version' : '1.1' , 
             'operation' : 'searchRetrieve' , 
             'query' : 'Klimawandel and jhr=2010', 
             'recordSchema' : 'MARC21-xml'} 

#Abfrage der Schnittstelle:
r = requests.get(dnb_url, params = parameter)
#Parsen der Antwort in xml:
response = soup(r.content, features="xml")
#Überführen der einzelnen, in der Antwort enthaltenen Datensätze/Treffer ('Records') in eine Liste:
result = response.find_all('record', {'type':'Bibliographic'})
print(response.prettify()[0:750])

<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  402
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    MARC21-xml
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
     <leader>
      00000pam a2200000 c 4500
     </leader>
     <controlfield tag="001">
      998800473
     </controlfield>
     <controlfield tag="003">
      DE-101
     </controlfield>
     <controlfield tag="005">
      20171202124616.0
     </controlfield>
     <controlfield tag="007">
      tu
     </controlfield>
     <controlfield tag="008">
      0912


In [7]:
number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')
print(len(result))

402 Ergebnisse
10


Wichtig: Die SRU-Schnittstelle liefert per default zunächst immer nur 10 Datensätze auf einmal aus, maximal 100.  

### Umwandlung in eine Funktion:

Funktion, die dank while-Schleife alle Datensätze der Anfrage sammelt:  

In [9]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    i = 0 
    records = []

    while i % 100 == 0: 
        params.update({'startRecord': i})       
        r = requests.get(base_url, params=params)
        xml = soup(r.content, features="xml")
        result = xml.find_all('record', {'type':'Bibliographic'})
        records+=result
        num_results = len(result)
        if num_results != 100:
            break
        else: 
            i += 100  
        
    return records

In [11]:
records = dnb_sru('Studentenbewegung') #and location=onlinefree and cod=pdf')
print(len(records), 'Ergebnisse')

992 Ergebnisse


In [13]:
print(records[137].prettify())

<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
 <leader>
  00000nam a2200000uc 4500
 </leader>
 <controlfield tag="001">
  1159579059
 </controlfield>
 <controlfield tag="003">
  DE-101
 </controlfield>
 <controlfield tag="005">
  20200423085446.0
 </controlfield>
 <controlfield tag="007">
  cr||||||||||||
 </controlfield>
 <controlfield tag="008">
  180515s2018    gw |||||o|||| 00||||ger
 </controlfield>
 <datafield ind1=" " ind2=" " tag="015">
  <subfield code="a">
   18,O06
  </subfield>
  <subfield code="2">
   dnb
  </subfield>
 </datafield>
 <datafield ind1="7" ind2=" " tag="016">
  <subfield code="2">
   DE-101
  </subfield>
  <subfield code="a">
   1159579059
  </subfield>
 </datafield>
 <datafield ind1=" " ind2=" " tag="020">
  <subfield code="a">
   9783406719721
  </subfield>
  <subfield code="9">
   978-3-406-71972-1
  </subfield>
 </datafield>
 <datafield ind1="3" ind2=" " tag="024">
  <subfield code="a">
   9783406719721
  </subfield>
 </datafield>
 

## Inhalte aus dem XML extrahieren: 

### Einblick: 

In [15]:
test_record = records[137]
test_record = str(test_record)
print(test_record)

<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
<leader>00000nam a2200000uc 4500</leader>
<controlfield tag="001">1159579059</controlfield>
<controlfield tag="003">DE-101</controlfield>
<controlfield tag="005">20200423085446.0</controlfield>
<controlfield tag="007">cr||||||||||||</controlfield>
<controlfield tag="008">180515s2018    gw |||||o|||| 00||||ger  </controlfield>
<datafield ind1=" " ind2=" " tag="015">
<subfield code="a">18,O06</subfield>
<subfield code="2">dnb</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="016">
<subfield code="2">DE-101</subfield>
<subfield code="a">1159579059</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="020">
<subfield code="a">9783406719721</subfield>
<subfield code="9">978-3-406-71972-1</subfield>
</datafield>
<datafield ind1="3" ind2=" " tag="024">
<subfield code="a">9783406719721</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="024">
<subfield code="2">urn</subfield>
<subfield code="a">urn:nbn:de:10

In [17]:
# Parsen des XML: 
root = etree.fromstring(test_record)
print(root)

<Element {http://www.loc.gov/MARC21/slim}record at 0x2357625ddc0>


In [19]:
# Extrahieren des Inhalts des Controlfields "001"
controlfield_001 = root.find(".//controlfield[@tag='001']")
print(controlfield_001)

None


In [21]:
# Definieren des Namespaces: 
namespace = {'marc': 'http://www.loc.gov/MARC21/slim'}

In [23]:
controlfield_001 = root.find('.//marc:controlfield[@tag="001"]', namespaces=namespace)#.text
print(controlfield_001)

<Element {http://www.loc.gov/MARC21/slim}controlfield at 0x23506b9d900>


In [25]:
datafield_245_a = root.xpath('.//marc:datafield[@tag="245"]/marc:subfield[@code="a"]/text()', namespaces=namespace)
print(datafield_245_a)

['\x98Das\x9c andere Achtundsechzig']


In [27]:
print(type(datafield_245_a))

<class 'list'>


#### Erstellen einer einfachen Funktion: 

In [29]:
def simple_parse_record(record):

    record = str(record).replace('\x98', '').replace('\x9c', '')
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
    meta_dict = {"idn":idn, "titel":titel}
    #print(meta_dict)
    
    return meta_dict

In [31]:
output = [simple_parse_record(record) for record in records]

In [32]:
df_simple = pd.DataFrame(output)
df_simple

,idn,titel
0,1337043966,Ausdrucksstarker Sprachgebrauch als Katalysato...
1,1325298123,Burg in der Brandung?
2,1338119109,Der Vietnamkrieg und die 68er Bewegung. Ursach...
3,1337965448,Der Vietnamkrieg und die 68er Bewegung. Ursach...
4,1334711879,Proteste homosexueller Studierender in Münster
...,...,...
987,1333866011,Rudi Dutschke: Tragische Folge eines Attentats
988,362816468,Stimmen aus der deutschen christlichen Student...
989,362816565,Stimmen und Zeugnisse
990,122532694X,Türkische und Türkisch-Deutsche Perspektiven i...


### Schönere, besser zu bedienende Funktion: 

In [37]:
def parse_record(record):
    
    record = str(record).replace('\x98', '').replace('\x9c', '')
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    def extract_text(xpath_query):
        fields = xml.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text for field in fields if field.text)
        return "unknown"

    
    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    author_gnd = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='0']")
    links = extract_text("marc:datafield[@tag='856']/marc:subfield[@code='u']")
    
    #add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    #add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")

    
    return {"idn": idn, "titel": titel, "author": author, "rela":author_rela, "gnd":author_gnd} #, "links":links}
                                    #"additional author":add_author, "added_rela": add_author_rela}

In [39]:
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

,idn,titel,author,rela,gnd
0,1337043966,Ausdrucksstarker Sprachgebrauch als Katalysato...,unknown,unknown,unknown
1,1325298123,Burg in der Brandung?,"Gilcher-Holtey, Ingrid",Verfasser,unknown
2,1338119109,Der Vietnamkrieg und die 68er Bewegung. Ursach...,"Schröders, Katharina",Verfasser,unknown
3,1337965448,Der Vietnamkrieg und die 68er Bewegung. Ursach...,"Schröders, Katharina",Verfasser,unknown
4,1334711879,Proteste homosexueller Studierender in Münster,"Schäfers, Norbert",Verfasser,unknown
...,...,...,...,...,...
987,1333866011,Rudi Dutschke: Tragische Folge eines Attentats,"Meißner, Thomas",Verfasser,unknown
988,362816468,Stimmen aus der deutschen christlichen Student...,unknown,unknown,unknown
989,362816565,Stimmen und Zeugnisse,unknown,unknown,unknown
990,122532694X,Türkische und Türkisch-Deutsche Perspektiven i...,"Gezen, Ela",Verfasser,unknown


## Speichern: 

In [48]:
df.to_csv("Studentenbewegung.csv", index=False, encoding="utf-8-sig")